In [121]:
import pandas as pd
import numpy as np

df = pd.read_csv('datasets/WEB_LOG_DATA.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5866 entries, 0 to 5865
Data columns (total 6 columns):
ip           5866 non-null object
date_time    5866 non-null object
request      5866 non-null object
step         5866 non-null int64
session      5866 non-null int64
user_id      5866 non-null int64
dtypes: int64(3), object(3)
memory usage: 275.0+ KB


In [122]:
df.head()

,ip,date_time,request,step,session,user_id
0,c210-49-32-6.rochd2.,18/Aug/2017:21:25:07,/,1,3,3
1,visp.inabox.telstra.,19/Aug/2017:08:24:28,/,1,12,12
2,adsl-61-95-54-84.requ,19/Aug/2017:08:33:01,/,1,13,13
3,d220-236-91-52.adsl.n,19/Aug/2017:09:16:06,/,1,15,15
4,allptrs.eq.edu.au,19/Aug/2017:09:47:54,/,1,22,22


In [123]:
df = df[df['request']!='/robots.txt']
df = df[df['request']!='/favicon.ico']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4917 entries, 0 to 5865
Data columns (total 6 columns):
ip           4917 non-null object
date_time    4917 non-null object
request      4917 non-null object
step         4917 non-null int64
session      4917 non-null int64
user_id      4917 non-null int64
dtypes: int64(3), object(3)
memory usage: 268.9+ KB


In [129]:
sessions = df.groupby(['user_id'])['request'].apply(list)
print(sessions.head(10))

user_id
2                         [/code/Global/code/menu.html]
3      [/, /guarantee.html, /more.html, /services.html]
5                            [/code/Ultra/services.htm]
6     [/richlands, /richlands/, /richlands/fileuploa...
7     [/newfarm, /newfarm/specials/, /richlands, /ri...
9                        [/code/Global/code/oform.html]
10    [/cgi-bin/FormMail.pl, /cgi-bin/FormMail.pl, /...
11    [/newfarm/, /richlands/, /richlands/contact/, ...
12                                                  [/]
13                                    [/, /direct.html]
Name: request, dtype: object


In [150]:
from apyori import apriori
# type cast the transactions from pandas into normal list format and run apriori
session_list = list(sessions)
results = list(apriori(session_list, min_support=0.03, min_confidence=0.1))
# print first 5 rules
print(results[:5])

[RelationRecord(items=frozenset({'/'}), support=0.4399271844660194, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/'}), confidence=0.4399271844660194, lift=1.0)]), RelationRecord(items=frozenset({'/newfarm/'}), support=0.15351941747572814, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/newfarm/'}), confidence=0.15351941747572814, lift=1.0)]), RelationRecord(items=frozenset({'/newfarm/javascript/menu.js'}), support=0.16868932038834952, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/newfarm/javascript/menu.js'}), confidence=0.16868932038834952, lift=1.0)]), RelationRecord(items=frozenset({'/newfarm/pricelist'}), support=0.10800970873786407, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/newfarm/pricelist'}), confidence=0.10800970873786407, lift=1.0)]), RelationRecord(items=frozenset({'/newfarm/pricelist/'}), support=0.1049757281553398, ordere

In [156]:
def convert_apriori_results_to_pandas_df(results):
    rules = []

    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
            rule_set.support, rule.confidence, rule.lift])

    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift'])
result_df = convert_apriori_results_to_pandas_df(results)
result_df.head(20)

,Left_side,Right_side,Support,Confidence,Lift
0,,/,0.439927,0.439927,1.000000
1,,/newfarm/,0.153519,0.153519,1.000000
2,,/newfarm/javascript/menu.js,0.168689,0.168689,1.000000
3,,/newfarm/pricelist,0.108010,0.108010,1.000000
4,,/newfarm/pricelist/,0.104976,0.104976,1.000000
5,,/services.html,0.141383,0.141383,1.000000
6,/direct.html,/,0.037621,0.704545,1.601505
7,/,/more.html,0.067961,0.154483,1.831565
8,/more.html,/,0.067961,0.805755,1.831565
9,/newfarm,/,0.035801,0.702381,1.596585


In [158]:
# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Lift', ascending=False)
result_df.head(50)

,Left_side,Right_side,Support,Confidence,Lift
57,/richlands/,/richlands,0.043083,0.910256,18.988640
56,/richlands,/richlands/,0.043083,0.898734,18.988640
109,"/newfarm/javascript/menu.js,/newfarm/specials/",/newfarm/specials,0.035194,0.753247,18.808343
59,/richlands/javascript/menu.js,/richlands/,0.036408,0.869565,18.372352
58,/richlands/,/richlands/javascript/menu.js,0.036408,0.769231,18.372352
64,"/newfarm/,/",/newfarm,0.035194,0.865672,16.983653
108,"/newfarm/javascript/menu.js,/newfarm/specials",/newfarm/specials/,0.035194,0.983051,16.701730
54,/newfarm/specials,/newfarm/specials/,0.038228,0.954545,16.217432
55,/newfarm/specials/,/newfarm/specials,0.038228,0.649485,16.217432
31,/newfarm/contact/,/newfarm/contact,0.043083,0.797753,14.939734


In [154]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 57 to 11
Data columns (total 5 columns):
Left_side     146 non-null object
Right_side    146 non-null object
Support       146 non-null float64
Confidence    146 non-null float64
Lift          146 non-null float64
dtypes: float64(3), object(2)
memory usage: 6.8+ KB
